## Bienvenido al maravilloso mundo del prueba error!! 

In [1]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
from sklearn import preprocessing


### Os presento H20, que te ayuda a predecir un modelo sin tener que ir toqueteando mil parametros


In [2]:
train=pd.read_csv("./Input/train.csv")
pred=pd.read_csv("./Input/predict.csv")

In [3]:
label = preprocessing.LabelEncoder()
train["cut"] = label.fit_transform(train["cut"])
train["color"] = label.fit_transform(train["color"])
train["clarity"] = label.fit_transform(train["clarity"])
train.head()
pred["cut"] = label.fit_transform(pred["cut"])
pred["color"] = label.fit_transform(pred["color"])
pred["clarity"] = label.fit_transform(pred["clarity"])
pred.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.33,4,5,1,62.0,58.0,4.44,4.46,2.76
1,1,1.21,4,0,3,62.4,58.0,6.77,6.83,4.24
2,2,1.06,4,0,2,59.3,60.0,6.64,6.71,3.96
3,3,0.36,2,1,6,61.4,57.0,4.64,4.61,2.54
4,4,0.70,2,1,4,62.3,54.0,5.67,5.72,3.55


In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "11.0.8" 2020-07-14 LTS; Java(TM) SE Runtime Environment 18.9 (build 11.0.8+10-LTS); Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.8+10-LTS, mixed mode)
  Starting server from /usr/local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/0v/jh6lh2p53ls9j3fnf4prlq7c0000gn/T/tmpsuogok50
  JVM stdout: /var/folders/0v/jh6lh2p53ls9j3fnf4prlq7c0000gn/T/tmpsuogok50/h2o_david_started_from_python.out
  JVM stderr: /var/folders/0v/jh6lh2p53ls9j3fnf4prlq7c0000gn/T/tmpsuogok50/h2o_david_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,"7 days, 17 hours and 8 minutes"
H2O_cluster_name:,H2O_from_python_david_fphjby
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [5]:
h2train = h2o.H2OFrame(train)
h2pred = h2o.H2OFrame(pred)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
train.columns

Index(['id', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y',
       'z', 'price'],
      dtype='object')

In [7]:
x = list(train.drop(columns = ['price', 'id'] ).columns)
y = 'price'

In [10]:
automl = H2OAutoML(max_runtime_secs = 999)
automl.train(x,y,training_frame = h2train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [11]:
leader_board = automl.leaderboard
leader_board.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201006_085524,285408,534.236,285408,268.701,0.0925949
StackedEnsemble_BestOfFamily_AutoML_20201006_085524,288795,537.396,288795,270.17,0.0918256
GBM_grid__1_AutoML_20201006_085524_model_4,298751,546.581,298751,272.445,0.090949
GBM_2_AutoML_20201006_085524,299076,546.878,299076,283.222,nan
GBM_3_AutoML_20201006_085524,302699,550.181,302699,281.403,nan
GBM_grid__1_AutoML_20201006_085524_model_5,304816,552.102,304816,274.768,0.0926815
GBM_1_AutoML_20201006_085524,305013,552.28,305013,289.289,0.10774
GBM_grid__1_AutoML_20201006_085524_model_6,305556,552.771,305556,282.794,0.103045
GBM_4_AutoML_20201006_085524,311349,557.986,311349,284.856,0.101595
XGBoost_grid__1_AutoML_20201006_085524_model_2,318712,564.546,318712,296.909,nan


In [12]:
predictions = automl.leader.predict(h2pred)
predictions.head()

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
730.437
6156.51
5829.89
1139.29
3192.88
542.487
1425.24
868.043
8884.54
682.193


In [40]:
sub = predictions.as_data_frame()
columns = ['id','price']
sub =sub.reset_index()
sub.columns = columns


### Guardamos nuestro modelo

In [41]:
sub.to_csv('modelh2o8.csv',index=False)